# Combine Token Level Results

In this notebook i combine token level results into a single table including the model used to paraphrase and score the n-grams. I then save a single table.

In [22]:
import pandas as pd

### Set locations and model types

This could potentially do with more work but works for now.

In [23]:
save_loc = '/Volumes/BCross/paraphrase examples slurm/Wiki-Test/combined_token_level_results_raw.xlsx'

gpt_base_loc = '/Volumes/BCross/paraphrase examples slurm/Wiki-Test'
modern_bert_base_loc = '/Volumes/BCross/paraphrase examples slurm/Wiki-Test/ModernBERT-base'

results_dict = [
    {"paraphrasing_model": "gpt5","scoring_model": "qwen"},
    {"paraphrasing_model": "gpt5","scoring_model": "gemma"},
    {"paraphrasing_model": "gpt5","scoring_model": "llama"},
    {"paraphrasing_model": "gpt5","scoring_model": "gpt2"},
    {"paraphrasing_model": "ModernBERT-base","scoring_model": "gpt2"},
]

### Combine the tables and add columns

In [24]:
combined_results_list = []

for result in results_dict:
    paraphrasing_model = result['paraphrasing_model']
    scoring_model = result['scoring_model']
    
    if paraphrasing_model == "gpt5":
        base_loc = gpt_base_loc
    else:
        base_loc = modern_bert_base_loc
        
    read_loc = f"{base_loc}/{scoring_model} results/token_level_results_inc_rank.xlsx"
    result_table = pd.read_excel(read_loc)
    
    result_table['paraphrasing_model'] = paraphrasing_model
    result_table['scoring_model'] = scoring_model
    
    combined_results_list.append(result_table)
    
results = (
    pd.concat(combined_results_list, ignore_index=True)
    .drop(columns=['index'])
    .reset_index()
)
front_cols = ['index', 'paraphrasing_model', 'scoring_model']
results = results[front_cols + [c for c in results.columns if c not in front_cols]]

In [26]:
results.head()

,index,paraphrasing_model,scoring_model,sample_id,problem,corpus,known_author,unknown_author,known_doc_id,unknown_doc_id,...,num_phrases,phrases_kept,llr_no_context,llr_known,llr_unknown,unknown_ref_vs_top_rank,normalised_llr_no_context,normalised_llr_known,normalised_llr_unknown,source_file
0,0,gpt5,qwen,1,Hodja_Nasreddin vs Hodja_Nasreddin,Wiki,Hodja_Nasreddin,Hodja_Nasreddin,hodja_nasreddin_text_1,hodja_nasreddin_text_3,...,1,1,0.311031,0.545463,0.503060,-0.636970,0.311031,0.545463,0.503060,hodja_nasreddin_text_1 vs hodja_nasreddin_text...
1,1,gpt5,qwen,1,Hodja_Nasreddin vs Hodja_Nasreddin,Wiki,Hodja_Nasreddin,Hodja_Nasreddin,hodja_nasreddin_text_1,hodja_nasreddin_text_3,...,7,7,9.261673,5.766482,4.629612,-7.716452,1.323096,0.823783,0.661373,hodja_nasreddin_text_1 vs hodja_nasreddin_text...
2,2,gpt5,qwen,1,Hodja_Nasreddin vs Hodja_Nasreddin,Wiki,Hodja_Nasreddin,Hodja_Nasreddin,hodja_nasreddin_text_1,hodja_nasreddin_text_3,...,12,12,18.661448,10.422070,9.763942,-14.550765,1.555121,0.868506,0.813662,hodja_nasreddin_text_1 vs hodja_nasreddin_text...
3,3,gpt5,qwen,4,Hodja_Nasreddin vs HonestopL,Wiki,Hodja_Nasreddin,HonestopL,hodja_nasreddin_text_1,honestopl_text_1,...,1,1,3.169289,0.684159,0.116089,0.000000,3.169289,0.684159,0.116089,hodja_nasreddin_text_1 vs honestopl_text_1.xlsx
4,4,gpt5,qwen,4,Hodja_Nasreddin vs HonestopL,Wiki,Hodja_Nasreddin,HonestopL,hodja_nasreddin_text_1,honestopl_text_1,...,4,4,7.598392,1.748907,1.306813,-0.649815,1.899598,0.437227,0.326703,hodja_nasreddin_text_1 vs honestopl_text_1.xlsx


### Save Results

In [28]:
results.to_excel(save_loc, index=False)